In [3]:
import json
import logging
import os
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from dotenv import load_dotenv

In [4]:

# Loading environmental variables
# Be sure to update your .env file to include your API Key
load_dotenv()

True

In [5]:

##### ENTER YOUR API KEY
API_KEY = os.getenv("API_KEY")

In [6]:
input_csv = "sample_addresses.csv"
df = pd.read_csv(input_csv)

In [7]:
df.head()

,user_id,address
0,1,0x6acdfba02d390b97ac2b2d42a63e85293bcc160e
1,2,0x7F19720A857F834887FC9A7bC0a0fBe7Fc7f8102
2,3,0x92193107FB10B3B372AB21cC90b5a4DBd67861d9
3,4,0xe1e8C9fa8F0cB41CcF9b92061b696EC247Fcc449
4,5,0xC5Ca16CD564eA101bBF2Dba1346499289F142B07


In [8]:
len(df)

11

In [9]:
# Define header JSON to be used in each API call.
headers = {"token": API_KEY, "Content-Type": "application/json"}


In [23]:
def address_screen_post(address):
    """
    POST
    Calls Address Screen API POST endpoint to register the address.
    """
    REGISTER_URL = "https://api.chainalysis.com/api/risk/v2/entities"
    newPayload = json.dumps({"address":address})
    r = requests.request("POST", REGISTER_URL, headers=headers, data=newPayload)
    return r

In [24]:
def address_screen_get(address):
    """
    GET
    Calls Address Screen API endpoint via GET to fetch info on an address.
    """  
    FETCH_URL = f"https://api.chainalysis.com/api/risk/v2/entities/{address}"
    response = requests.request("GET", FETCH_URL, headers=headers)
    return json.loads(response.text)

In [26]:
TEST_ADDRESS = '0x7F19720A857F834887FC9A7bC0a0fBe7Fc7f8102'

In [27]:
address_screen_post(TEST_ADDRESS)

<Response [201]>

In [32]:
# ITERATE
# Iterate over each row of the CSV file and call the Address Screening API.
# https://docs.chainalysis.com/api/address-screening/#register-an-address
responses = []
print("Registering and evaluating addresses...")
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    address = row["address"]

    address_screen_post(address)
    if address_screen_get(address)['status'] == 'COMPLETE':
        continue

    
    

logging.info("All API calls finished.")


Registering and evaluating addresses...


100%|██████████| 11/11 [00:21<00:00,  1.95s/it]


In [33]:

# In order to correctly flatten the JSON, we need to alter the JSON for those responses where `addressIdentifications` is an empty list. We simply add an empty dict.
# more info: https://stackoverflow.com/questions/63813378/how-to-json-normalize-a-column-in-pandas-with-empty-lists-without-losing-record/63876897#63876897
# Flattening
for i, d in enumerate(responses):
    if not d["addressIdentifications"]:
        responses[i]["addressIdentifications"] = [{}]

# Dropping type None, which represent errored API responses.
data = []
for i in responses:
    if i is not None:
        data.append(i)


In [34]:
len(data)

0

# Parsing `data`

In [35]:
exp = pd.json_normalize(
    data,
    meta=[
        "address",
        "risk",
        ["exposures"],
        ["cluster", "name"],
        ["cluster", "category"]
    ],
    record_path="exposures",
    errors="ignore",
)

clusters = exp[
    ["address"
    ,"cluster.name"
    ,"cluster.category"]
]

exp_pivot = pd.pivot_table(exp,index='address',columns=['category'])

exp_pivot



KeyError: "None of [Index(['address', 'cluster.name', 'cluster.category'], dtype='object')] are in the [columns]"

In [36]:

tmp_cols = ["address"]
for i in exp_pivot.columns.tolist():
    tmp_cols.append(i[1])

exp_pivot = pd.DataFrame(exp_pivot.to_records())
exp_pivot.columns = tmp_cols
exp_pivot.fillna(0,inplace=True)

exp_pivot

NameError: name 'exp_pivot' is not defined

In [208]:
addressIdentifications = pd.json_normalize(
    data,
    meta=["address", ["addressIdentifications"]],
    record_path="addressIdentifications",
    record_prefix="addressId_",
)

addressIdentifications

,addressId_name,addressId_category,addressId_description,address,addressIdentifications
0,NaN,NaN,NaN,0xd5f53937886be426b9227d4413f5aea08062c145,{}
1,NaN,NaN,NaN,0xd35357e58daeaec732929da579e812c9ac216cc6,{}
2,NaN,NaN,NaN,0x9606d3ce1dd49de880344ecf5d31509d9ddf82b1,{}
3,NaN,NaN,NaN,0x991aa52fc5f3ba1644e272f712abd4b7b2d0b577,{}
4,NaN,NaN,NaN,0x5f0166092306a8215300e3a3b4e1f17d7f46cd6c,{}
...,...,...,...,...,...
756,NaN,NaN,NaN,0x471ece3750da237f93b8e339c536989b8978a438,{}
757,NaN,NaN,NaN,0x5bd5f2bc18bd01d50512cf2bd269d789d1f6664a,{}
758,NaN,NaN,NaN,0xd434b34c05e6b11fb356ef85f6cc7823f0632dbc,{}
759,NaN,NaN,NaN,0xce79dae41cb9c68ed3c6fbcf1a748e34e780cde5,{}


In [209]:
risk = pd.DataFrame(data=data,columns=['address','risk'])

risk

,address,risk
0,0xd5f53937886be426b9227d4413f5aea08062c145,Low
1,0xd35357e58daeaec732929da579e812c9ac216cc6,Low
2,0x9606d3ce1dd49de880344ecf5d31509d9ddf82b1,Low
3,0x991aa52fc5f3ba1644e272f712abd4b7b2d0b577,Low
4,0x5f0166092306a8215300e3a3b4e1f17d7f46cd6c,Low
...,...,...
756,0x471ece3750da237f93b8e339c536989b8978a438,Low
757,0x5bd5f2bc18bd01d50512cf2bd269d789d1f6664a,Low
758,0xd434b34c05e6b11fb356ef85f6cc7823f0632dbc,Low
759,0xce79dae41cb9c68ed3c6fbcf1a748e34e780cde5,Low


In [210]:
# Setting columns to fill NA with 0
exposure_cols = exp_pivot.columns.tolist()
exposure_cols = exposure_cols[1:]

In [212]:
tmp_summary = pd.merge(left=addressIdentifications,right=exp_pivot,on='address', how = 'outer')

tmp_summary2 = pd.merge(left=tmp_summary,right=clusters,on='address',how='left')

summary = pd.merge(left=tmp_summary2,right=risk,on='address',how='left')

summary = summary.drop_duplicates(subset='address',keep='first')

summary[exposure_cols] = summary[exposure_cols].fillna(0)

summary = pd.merge(left=summary,right=df,on='address')

summary

,addressId_name,addressId_category,addressId_description,address,addressIdentifications,atm,child abuse material,darknet market,decentralized exchange contract,exchange,...,terrorist financing,token smart contract,unnamed service,untraced,cluster.name,cluster.category,risk,Entity,SheetOrigin,FromAddress
0,NaN,NaN,NaN,0xd5f53937886be426b9227d4413f5aea08062c145,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,AP Grant Foundation,summary,anchorage-grantco-vault-1
1,NaN,NaN,NaN,0xd35357e58daeaec732929da579e812c9ac216cc6,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,AP Grant Foundation,summary,anchorage-grantco-vault-1
2,NaN,NaN,NaN,0x9606d3ce1dd49de880344ecf5d31509d9ddf82b1,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,AP Grant Foundation,summary,anchorage-grantco-vault-2
3,NaN,NaN,NaN,0x991aa52fc5f3ba1644e272f712abd4b7b2d0b577,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,AP Grant Foundation,summary,anchorage-grantco-vault-2
4,NaN,NaN,NaN,0x5f0166092306a8215300e3a3b4e1f17d7f46cd6c,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,AP Grant Foundation,summary,anchorage-grantco-vault-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,NaN,NaN,NaN,0x471ece3750da237f93b8e339c536989b8978a438,{},0.0,0.0,0.0,0.00,15.720,...,0.0,0.0,0.0,0.0,None,None,Low,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2
757,NaN,NaN,NaN,0x5bd5f2bc18bd01d50512cf2bd269d789d1f6664a,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2
758,NaN,NaN,NaN,0xd434b34c05e6b11fb356ef85f6cc7823f0632dbc,{},0.0,0.0,0.0,0.00,0.000,...,0.0,0.0,0.0,0.0,NaN,NaN,Low,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2
759,NaN,NaN,NaN,0xce79dae41cb9c68ed3c6fbcf1a748e34e780cde5,{},0.0,0.0,0.0,62.79,451.587,...,0.0,0.0,0.0,0.0,None,None,Low,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2


In [215]:
columns = [
    'address',
    'Entity',
 'SheetOrigin',
 'FromAddress',
 'risk',
 'cluster.name',
 'cluster.category',
 'addressId_name',
 'addressId_category',
 'addressId_description',
 'atm',
 'child abuse material',
 'darknet market',
 'decentralized exchange contract',
 'exchange',
 'fees',
 'fraud shop',
 'gambling',
 'high risk exchange',
 'high risk jurisdiction',
 'hosted wallet',
 'ico',
 'illicit actor-org',
 'infrastructure as a service',
 'lending contract',
 'merchant services',
 'mining',
 'mining pool',
 'mixing',
 'no category',
 'other',
 'p2p exchange',
 'protocol privacy',
 'ransomware',
 'sanctions',
 'scam',
 'smart contract',
 'stolen funds',
 'terrorist financing',
 'token smart contract',
 'unnamed service',
 'untraced'
 ]

In [216]:
summary[columns]

,address,Entity,SheetOrigin,FromAddress,risk,cluster.name,cluster.category,addressId_name,addressId_category,addressId_description,...,protocol privacy,ransomware,sanctions,scam,smart contract,stolen funds,terrorist financing,token smart contract,unnamed service,untraced
0,0xd5f53937886be426b9227d4413f5aea08062c145,AP Grant Foundation,summary,anchorage-grantco-vault-1,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
1,0xd35357e58daeaec732929da579e812c9ac216cc6,AP Grant Foundation,summary,anchorage-grantco-vault-1,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
2,0x9606d3ce1dd49de880344ecf5d31509d9ddf82b1,AP Grant Foundation,summary,anchorage-grantco-vault-2,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
3,0x991aa52fc5f3ba1644e272f712abd4b7b2d0b577,AP Grant Foundation,summary,anchorage-grantco-vault-2,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
4,0x5f0166092306a8215300e3a3b4e1f17d7f46cd6c,AP Grant Foundation,summary,anchorage-grantco-vault-2,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,0x471ece3750da237f93b8e339c536989b8978a438,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2,Low,None,None,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
757,0x5bd5f2bc18bd01d50512cf2bd269d789d1f6664a,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
758,0xd434b34c05e6b11fb356ef85f6cc7823f0632dbc,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2,Low,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0
759,0xce79dae41cb9c68ed3c6fbcf1a748e34e780cde5,cLabs Inc.,summary,Celo Labs - Anchorage Treasury 2,Low,None,None,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,126.20001,0.0,0.0,0.0,0.0,0.0


In [217]:
summary[columns].to_csv("Chainalysis_AddressScreenAPI_CELO.csv",index=False)

----

----

# Searching address participation on different blockchains

Using Etherscan API (https://docs.etherscan.io/api-endpoints/accounts), can we see which addresses in the population actually have exposure to ETH L1 and which ones are only native to CELO?

In [63]:
ETHERSCAN_API_KEY = "TSY3BYQSKR99RWUAKVJPZWWTAXZVSBNGMG"

In [ ]:
ETHERSCAN_URL = "https://api.etherscan.io/api?module=account&action=balance&address=0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae&tag=latest&apikey=YourApiKeyToken"

In [65]:
addy = "0xf01572bd63fc74b6225fe98be5454ff4284b850b"
r = requests.get(
    f"https://api.etherscan.io/api?module=account&action=balance&address={addy}&tag=latest&apikey=TSY3BYQSKR99RWUAKVJPZWWTAXZVSBNGMG"
)

r.text

'{"status":"1","message":"OK","result":"0"}'